In [ ]:
excludeDesc_columns = ['patid', 'practice_id', #identifier
                   'BMI', #use the categorical instead
                   'ICS_medication_possesion_ratio', #the max value is inf
                   'Spacer', 'numPCSAsthma',  #all zero
                   'outcome_6months', 'outcome_9months', 'outcome_12months', 'outcome_15months', 'outcome_18months',
                   'outcome_21months', 'outcome_24months', 'outcome_combined_9months', 
                   'outcome_combined_15months', 'outcome_combined_18months', 'outcome_combined_24months', #outcomes variable
                   'Country',#used for train-tes split only
                  ]
excludeDesc_columns = excludeDesc_columns + [x for x in features_columns if '_count' in x]
descData = masterData[masterData.columns.difference(excludeDesc_columns)]

In [ ]:
cat_vars = summaryData[summaryData['max'] <= 5].index.to_list() 
cat_vars.remove('numHospEvents')
cat_vars = cat_vars + categoricalNonnumericVars
cont_vars = summaryData[summaryData['max'] > 5].index.to_list() + ['numHospEvents']

In [ ]:
# writer = pd.ExcelWriter('../../code/descriptive_cat.xlsx', engine='xlsxwriter')
outcomes = ['outcome_combined_12months']
# cat_vars = ['gender_x', 'language', 'marital_status', 'ethnicity', 'admission_location', 'discharge_location']
for target_outcome in outcomes:
    desc_table = []
    print(target_outcome)
    for var in cat_vars:
        chi,pval,df,tab = chi2_contingency(pd.crosstab(descData[var].values, descData[target_outcome].values).apply(lambda c: c/c.sum(), axis=0))
        desc_table.append((var  + ' (n, % of total)','-' ,'-'))
        for group in descData[var].unique():
            noAsthma = descData[(descData[var]==group)&(descData[target_outcome]==0)].shape[0]
            noAsthmaPercent = round(noAsthma/sum(descData[target_outcome]==0)*100,2)
            asthma = descData[(descData[var]==group)&(descData[target_outcome]==1)].shape[0]
            asthmaPercent = round(asthma/sum(descData[target_outcome]==1)*100,2)
            desc_table.append((group, str(noAsthma) + ' (' + str(noAsthmaPercent) + '%)' , str(asthma) + ' (' + str(asthmaPercent) + '%)'))
    descriptive_cat = pd.DataFrame(desc_table, columns=['var','No asthma attack', 'Asthma attack'])
    # descriptive_cat.to_excel(writer, sheet_name=target_outcome)
    print('writing to Excel done!!')
# writer.save()

In [ ]:

for target_outcome in outcomes:
    desc_table_cont = []
    print(target_outcome)
    for var in descData.columns:
        if (var in cont_vars):
            tval,pval = ttest_ind(descData[var],descData[target_outcome])
            noAsthmaMean = np.round(np.mean(descData[var][descData[target_outcome]==0]),2)
            noAsthmaSD = np.round(np.std(descData[var][descData[target_outcome]==0]),2)
            asthmaMean = np.round(np.mean(descData[var][descData[target_outcome]==1]),2)
            asthmaSD = np.round(np.std(descData[var][descData[target_outcome]==1]),2)
            desc_table_cont.append((var  + ' (mean, std)', str(noAsthmaMean)  + ' (' + str(noAsthmaSD) + ')', str(asthmaMean) + ' (' + str(asthmaSD) + ')'))
    descriptive_cont = pd.DataFrame(desc_table_cont, columns=['var','No asthma attack', 'Asthma attack'])
    # descriptive_cont.to_excel(writer, sheet_name=target_outcome)
    print('writing to Excel done!!')
# writer.save()

In [ ]:
pd.concat([descriptive_cat, descriptive_cont]).to_csv(target_outcome + '.csv', index_label=False, index=False)